In [ ]:
import os

if "notebooks" in os.getcwd():
    # Change the current working directory
    os.chdir("..")

### Test Function Calling

In [ ]:
import json
from openai.types.chat import ChatCompletionMessage
from model.conversation import Conversation
from modules.chatbot import ChatBot
import modules.question_manager as ques_manager

save_questions_description = {
  "name": "save_questions",
  "description": "Set daily journal questions for the user. The questions can be for a Gratitude journal (What are you grateful for?), self-improvement (what charged you with energy, what drained you of energy, what would you like to do more, what would you like to do less), or anything else the user wants as a goal.",
  "parameters": {
    "type": "object",
    "properties": {
      "questions": {
        "type": "array",
        "items": {
          "type": "string"
        },
        "description": "array of strings, representing the questions the user wants to set as their daily questions"
      }
    },
    "required": [
      "questions"
    ]
  }
}

chatbot = ChatBot(function_descriptions=[save_questions_description])
conversation = Conversation(chatbot.system_message)

conversation.add_turn("user", "Can you add the following questions as my daily questions: 1. What made me happy, 2. What made me sad")

response: ChatCompletionMessage = chatbot.chat(conversation)
if response.function_call:
  if response.function_call.name == "save_questions":
    questions = json.loads(response.function_call.arguments)['questions']
    ques_manager.save_questions("test", questions)
    print("got here")
print(response)

### Make a title

In [6]:
multiline_questions = '\n * '.join(questions)
ai_answer = f"Do you want to save these as your daily questions?\n * {multiline_questions}"
conversation.add_turn("assistant", ai_answer)
conversation.add_turn("user", "yes")
conversation.add_turn(
    "function",
    "Successfully saved.",
    "save_questions"
)

In [ ]:
title = chatbot.get_title(conversation)
print(title)

### Make a summary